In [1]:
import cv2
import os
import json
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from lime.lime_image import LimeImageExplainer
from skimage.segmentation import mark_boundaries

import torch
import torch.nn.functional as F
from torchvision import models, transforms

In [2]:
# 图像分类模型
model = models.vgg19(pretrained=True)
model = model.eval()

d:\Anaconda\envs\explain\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
d:\Anaconda\envs\explain\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# 图像预处理
transform = transforms.Compose([
 transforms.Resize(256),
 transforms.CenterCrop(224),
 transforms.ToTensor()
])

transform_normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)
# 读取索引对应的类别
labels_path = '../imagenet_class_index.json'
with open(labels_path) as json_data:
    idx_to_labels = json.load(json_data)

def predict(imgs):
    '''
    img:将要预测的图像
    '''
    result = []
    for img in imgs:
        # 转换图像格式
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        transformed_img = transform(img)
        img = transform_normalize(transformed_img)
        img = img.unsqueeze(0)
        prediction_score, pred_label_idx = torch.topk(F.softmax(model(img), dim=1), 1)
        pred_label_idx.squeeze_()
        predicted_label = idx_to_labels[str(pred_label_idx.item())][1]
        #print('Predicted:', predicted_label, '(', prediction_score.squeeze().item(), ')')
        result.append(np.array([pred_label_idx, prediction_score.squeeze().item()]))
        #return [(i,result[i]) for i in range(len(result))]
    return result

In [4]:
input_path  = "../Images/"
output_path = "../Results/LIME"
if not os.path.isdir(output_path):
    os.makedirs(output_path)
for image_path in os.listdir(input_path):
    explainer = LimeImageExplainer()
    explanation = explainer.explain_instance(
        cv2.imread(os.path.join(input_path, image_path)),
        predict, # classification function
        top_labels=50,
        hide_color=0,
        num_samples=100)
    #temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=5, hide_rest=False)
    img_boundry1 = mark_boundaries(temp/255.0, mask)
    cv2.imwrite(os.path.join(output_path, './result_vgg19_'+image_path), img_boundry1*255)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [5]:
# 图像分类模型
model = models.resnet50(pretrained=True)
model = model.eval()

d:\Anaconda\envs\explain\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
d:\Anaconda\envs\explain\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
input_path  = "../Images/"
output_path = "../Results/LIME"
if not os.path.isdir(output_path):
    os.makedirs(output_path)
for image_path in os.listdir(input_path):
    explainer = LimeImageExplainer()
    explanation = explainer.explain_instance(
        cv2.imread(os.path.join(input_path, image_path)),
        predict, # classification function
        top_labels=50,
        hide_color=0,
        num_samples=100)
    #temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=5, hide_rest=False)
    img_boundry1 = mark_boundaries(temp/255.0, mask)
    cv2.imwrite(os.path.join(output_path, './result_resnet_'+image_path), img_boundry1*255)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]